In [ ]:
import numpy as np

def softmax(x):
    """Softmax function for vector x"""
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

def train_softmax_regression(X, y, num_classes, max_iters=100, tol=1e-5):
    """
    Trains a softmax regression model using the Newton-Raphson method.

    Parameters:
    - X: feature matrix of shape (num_data, num_features)
    - y: target class vector of shape (num_data,)
    - num_classes: number of classes
    - max_iters: maximum number of iterations (default: 100)
    - tol: convergence tolerance (default: 1e-5)

    Returns:
    - W: learned weights of shape (num_classes, num_features)
    - b: learned biases of shape (num_classes,)
    """

    # One-hot encode target classes
    y_onehot = np.eye(num_classes)[y]

    # Initialize weights and biases to zero
    num_data, num_features = X.shape
    W = np.zeros((num_classes, num_features))
    b = np.zeros(num_classes)

    # Train model using Newton-Raphson method
    for i in range(max_iters):

        # Compute probabilities using softmax function
        logits = np.dot(X, W.T) + b
        probs = np.apply_along_axis(softmax, 1, logits)

        # Compute gradient and Hessian of negative log-likelihood
        grad_J_W = 1/num_data * np.dot((probs - y_onehot).T, X)
        grad_J_b = 1/num_data * np.sum(probs - y_onehot, axis=0)

        Hessian_J_W = 1/num_data * np.zeros((num_features, num_classes, num_features))
        Hessian_J_b = 1/num_data * np.zeros((num_classes, num_classes))

        for j in range(num_classes):
            probs_j = probs[:, j]
            Hessian_J_W[j] = np.dot((probs_j * (1 - probs_j) * X.T), X)
            Hessian_J_b[j, j] = np.sum(probs_j * (1 - probs_j))

        # Apply Newton-Raphson update
        W_new = W - np.linalg.inv(Hessian_J_W.sum(axis=0)).dot(grad_J_W.T)
        b_new = b - np.linalg.inv(Hessian_J_b).dot(grad_J_b)

        # Check for convergence
        if np.max(np.abs(W_new - W)) < tol and np.max(np.abs(b_new - b)) < tol:
            W = W_new
            b = b_new
            break

        # Update weights and biases
        W = W_new
        b = b_new

    return W, b

# # Generate some random data
# X = np.random.randn(3, 4)
# y = np.array([0, 1, 2])

# # Train softmax regression model
# num_classes = len(np.unique(y))
# W, b = train_softmax_regression(X, y, num_classes)


In [ ]:
import numpy as np

# Generate random data
np.random.seed(123)
X = np.random.randn(100, 5)
y = np.random.randint(5, size=100)

# Train softmax regression model
W, b = train_softmax_regression(X, y, num_classes=5)

# Print learned weights and biases
print("Weights:\n", W)
print("Biases:\n", b)
X.shape


Weights:
 [[-0.02598701 -0.0365298   0.07210556  0.00702519 -0.01661395]
 [ 0.04208739 -0.07017528 -0.04297361  0.00232457  0.06873693]
 [-0.02376718  0.00071733  0.0726625  -0.08490233  0.03528968]
 [-0.0428995   0.00247003 -0.01639844  0.00863375  0.04819415]
 [ 0.03518724  0.01073504 -0.0116562   0.00514174 -0.03940783]]
Biases:
 [ 0.13609204 -0.23515828 -0.1120175   0.13529996  0.0365125 ]


(100, 5)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape

(120, 4)

In [ ]:
W, b = train_softmax_regression(X_train, y_train, 4)
probs = np.apply_along_axis(softmax, 1, np.dot(X_test, W.T) + b)
y_pred = np.argmax(probs, axis=1)
acc = np.mean(y_pred == y_test)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 63.33%


Work in Progress

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# Define the softmax function
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

# Define the gradient of the softmax function
def softmax_grad(x):
    p = softmax(x)
    return p * (1 - p)

# Define the Newton-Raphson update rule
def newton_update(X, y, w):
    p = softmax(X @ w)
    grad = X.T @ (p - y)
    hessian = X.T @ np.diag(p*(1-p)) @ X
    w_new = w - np.linalg.inv(hessian) @ grad
    return w_new

# Load the Iris dataset and standardize the features
iris = load_iris()
X = StandardScaler().fit_transform(iris.data)
y = np.eye(3)[iris.target]

# Initialize the weights
n_features = X.shape[1]
n_classes = y.shape[1]
w = np.zeros((n_features, n_classes))

# Train the model using the Newton-Raphson method
for i in range(50):
    w = newton_update(X, y, w)

# Make predictions and compute the accuracy
p = softmax(X @ w)
predictions = np.argmax(p, axis=1)
accuracy = np.mean(predictions == iris.target)
print("Accuracy: {:.2f}%".format(accuracy * 100))


ValueError: ignored

In [ ]:
import numpy as np

def softmax(theta, x):
    """Softmax function for a sample x with weight vector theta."""
    e = np.exp(theta.dot(x))
    return e / e.sum()

def neg_log_likelihood(theta, X, y):
    """Negative log-likelihood function for weight vector theta."""
    n = X.shape[0]
    k = theta.shape[0] // X.shape[1]
    J = 0
    for i in range(n):
        for j in range(k):
            J -= y[i,j] * np.log(softmax(theta[j*X.shape[1]:(j+1)*X.shape[1]], X[i]))
    return J / n

def gradient(theta, X, y):
    """Gradient of the negative log-likelihood function for weight vector theta."""
    n = X.shape[0]
    k = theta.shape[0] // X.shape[1]
    grad = np.zeros_like(theta)
    for i in range(n):
        for j in range(k):
            grad[j*X.shape[1]:(j+1)*X.shape[1]] -= X[i] * (y[i,j] - softmax(theta[j*X.shape[1]:(j+1)*X.shape[1]], X[i]))
    return grad / n

def hessian(theta, X, y):
    """Hessian matrix of the negative log-likelihood function for weight vector theta."""
    n = X.shape[0]
    k = theta.shape[0] // X.shape[1]
    hess = np.zeros((theta.shape[0], theta.shape[0]))
    for i in range(n):
        s = softmax(theta[0:X.shape[1]], X[i])
        hess[0:X.shape[1], 0:X.shape[1]] += np.outer(s, s)
        for j in range(1, k):
            s = softmax(theta[j*X.shape[1]:(j+1)*X.shape[1]], X[i])
            hess[j*X.shape[1]:(j+1)*X.shape[1], j*X.shape[1]:(j+1)*X.shape[1]] += np.outer(s, s)
    for j in range(k):
        hess[j*X.shape[1]:(j+1)*X.shape[1], j*X.shape[1]:(j+1)*X.shape[1]] -= np.diag(softmax(theta[j*X.shape[1]:(j+1)*X.shape[1]], X).flatten())
    return hess / n

def newton_raphson(X, y, max_iters=100, tol=1e-6):
    """Newton-Raphson method for softmax regression."""
    n = X.shape[0]
    k = y.shape[1]
    theta = np.zeros(X.shape[1] * k)
    for i in range(max_iters):
        J = neg_log_likelihood(theta, X, y)
        grad = gradient(theta, X, y)
        hess = hessian(theta, X, y)
        theta_new = theta - np.linalg.inv(hess).dot(grad)
        if np.abs(J - neg_log_likelihood(theta_new, X, y)) < tol:
            return theta_new
        theta = theta_new
    return theta


In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
iris = load_iris()
X = iris.data
y = iris.target
scaler = StandardScaler()
X = scaler.fit_transform(X)
y_onehot = np.zeros((y.size, y.max() + 1))
y_onehot[np.arange(y.size), y] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=0)
theta = newton_raphson(X_train, y_train)
y_pred = np.argmax(np.dot(X_test, theta.reshape((-1, X_train.shape[1])).T), axis=1)
accuracy = np.mean(y_pred == np.argmax(y_test, axis=1))
print("Accuracy:", accuracy)

ValueError: ignored

In [ ]:
import numpy as np

def softmax(X, theta):
    """Compute the softmax of the scores for each example in X."""
    scores = np.dot(X, theta)
    exp_scores = np.exp(scores)
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

def newton_raphson(X, y):
    """Compute the optimal weight vector for the softmax function using Newton-Raphson."""
    num_features = X.shape[1]
    num_classes = y.shape[1]
    theta = np.zeros(num_features * num_classes)
    for i in range(10):
        # Compute the softmax probabilities
        probs = softmax(X, theta)

        # Compute the gradient of the log-likelihood
        error = y - probs
        grad = -np.dot(X.T, error.flatten())

        # Compute the Hessian of the log-likelihood
        hess = np.zeros((num_features * num_classes, num_features * num_classes))
        for j in range(num_classes):
            hess_j = np.dot(X.T, (probs[:, j] * (1 - probs[:, j]))[:, None] * X)
            hess[j*num_features:(j+1)*num_features, j*num_features:(j+1)*num_features] = hess_j
        hess_inv = np.linalg.inv(hess)

        # Update the weight vector
        delta = np.dot(hess_inv, grad)
        theta -= delta

    return theta

# Simulate data with 3 classes and 4 features
np.random.seed(0)
X = np.random.randn(120, 4)
y = np.random.randint(0, 3, size=120)
y_onehot = np.zeros((y.size, y.max() + 1))
y_onehot[np.arange(y.size), y] = 1

# Run the Newton-Raphson algorithm on the simulated data
theta = newton_raphson(X, y_onehot)

# Compute the predicted class probabilities for the first 10 examples
probs = softmax(X[:10, :], theta)
print(probs)


ValueError: ignored